In [1]:
from pykalman import KalmanFilter
import pandas as pd
import numpy as np
import schedule
import os
import math
import matplotlib.pyplot as plt
import json
from shapely.geometry import Point, Polygon
import plotly.express as px
from geopy import Nominatim
from pygeocoder import Geocoder
import time
from sklearn.cluster import KMeans
geolocator = Nominatim(user_agent='Pechino.ipynb')

In [2]:
#importing the dataset of beijing
allBeijing = pd.read_csv('allPechino2.csv')
allBeijing

,Unnamed: 0,taxi_id,date_time,longitude,latitude,Distances_m,IntervalRecordTime_s,speed_km
0,0,1120,2008-02-02 13:37:15,116.41442,39.85614,353.2104719358517,302.0,4.210456
1,1,1120,2008-02-02 13:42:17,116.41041,39.85536,879.8997173829874,209.0,15.156167
2,2,1120,2008-02-02 13:45:46,116.41561,39.86219,218.39355397692162,92.0,8.545835
3,3,1120,2008-02-02 13:47:18,116.41639,39.86406,780.3486396214676,302.0,9.302169
4,4,1120,2008-02-02 13:52:20,116.41818,39.85718,62.355268461881536,52.0,4.316903
...,...,...,...,...,...,...,...,...
1028277,114214,8857,2008-02-08 17:16:52,116.46601,39.97386,0.0,300.0,0.000000
1028278,114215,8857,2008-02-08 17:21:52,116.46601,39.97386,1459.9758163690663,300.0,17.519710
1028279,114216,8857,2008-02-08 17:26:52,116.46516,39.98697,1446.4525177008009,300.0,17.357430
1028280,114217,8857,2008-02-08 17:31:52,116.47547,39.97664,1250.620746914865,344.0,13.087892


In [3]:

#Quello che bisogna fare adesso è prendere le coordinate di Pechino e utilizzare la libreria python-geohash per
#creare la griglia


In [4]:
#prenderò un area di 5 caratteri -> 5 ≤ 4.89km x4.89km


In [5]:
from polygeohasher import polygeohasher
import geopandas as gpd

gdf = gpd.read_file('./Beijing-Boundaries.geojson')


In [6]:
primary_df = polygeohasher.create_geohash_list(gdf,5,inner=False)

In [7]:
#secondary_df = polygeohasher.geohash_optimiser(primary_df,3,4,4)

In [8]:
#polygeohasher.optimization_summary(primary_df,secondary_df)

In [9]:
geo_df = polygeohasher.geohashes_to_geometry(primary_df,"geohash_list")
geo_df.to_file("PechinoGrid.geojson",driver = "GeoJSON")

In [10]:
PechinoContorno = json.load(open('PechinoGrid.geojson','r'))


In [11]:
import folium
m = folium.Map([39.80, 116.26], zoom_start=12, tiles='cartodbpositron')
folium.GeoJson(PechinoContorno, name="geojson").add_to(m)
m

In [12]:
geo_df

,geohash_list,geometry
0,wx4es,"POLYGON ((116.19141 39.99023, 116.23535 39.990..."
0,wx4f8,"POLYGON ((116.36719 39.81445, 116.41113 39.814..."
0,wx5n2,"POLYGON ((116.71875 40.47363, 116.76270 40.473..."
0,wx5r5,"POLYGON ((117.20215 40.60547, 117.24609 40.605..."
0,wx5rx,"POLYGON ((117.37793 40.69336, 117.42188 40.693..."
...,...,...
0,wx43m,"POLYGON ((115.88379 39.59473, 115.92773 39.594..."
0,wx41j,"POLYGON ((115.53223 39.55078, 115.57617 39.550..."
0,wx4f3,"POLYGON ((116.41113 39.77051, 116.45508 39.770..."
0,wx4zk,"POLYGON ((116.54297 40.64941, 116.58691 40.649..."


In [13]:
#problema con gli indici del geodataframe
#soluzione: copiarlo in un file csv, leggerlo in un dataframe, cancellare la colonna con indici errati ed infine copiare in un geodataframe
geo_df.to_csv("Tassellation.csv")

In [14]:
tasselation = pd.read_csv("./Tassellation.csv")

In [15]:
del tasselation["Unnamed: 0"]

In [16]:
tasselation

,geohash_list,geometry
0,wx4es,"POLYGON ((116.19140625 39.990234375, 116.23535..."
1,wx4f8,"POLYGON ((116.3671875 39.814453125, 116.411132..."
2,wx5n2,"POLYGON ((116.71875 40.4736328125, 116.7626953..."
3,wx5r5,"POLYGON ((117.2021484375 40.60546875, 117.2460..."
4,wx5rx,"POLYGON ((117.3779296875 40.693359375, 117.421..."
...,...,...
1098,wx43m,"POLYGON ((115.8837890625 39.5947265625, 115.92..."
1099,wx41j,"POLYGON ((115.5322265625 39.55078125, 115.5761..."
1100,wx4f3,"POLYGON ((116.4111328125 39.7705078125, 116.45..."
1101,wx4zk,"POLYGON ((116.54296875 40.6494140625, 116.5869..."


In [17]:
from shapely import wkt

tasselation['geometry'] = tasselation['geometry'].apply(wkt.loads)

In [18]:
geo_tassellation = gpd.GeoDataFrame(
    tasselation,geometry='geometry')
#risolto

In [19]:
#Adesso quello che bisogna fare è vedere quanti punti di tutto il dataframe ricadono all'interno delle celle

latlonPechinoLista=allBeijing[['longitude','latitude']].to_numpy()
#Lista di tutte le coordinate del dataframe

In [20]:
#Funzione che mi permette di verifcare se un determinato punto si trova in un'area

#posso evitare di utilizzare questa funzione per i punti lat lon
def checkBox(latlonPechinoLista,geo_tassellation):
    i = 0
    find = 0
    j=0
    result =[]
    while(j<int(latlonPechinoLista.size/2)):
        i=0
        k=len(geo_tassellation['geometry'])-1
        find=0
        while(i<len(geo_tassellation['geometry']) and k>i and find == 0):
            point= Point(latlonPechinoLista[j][0],latlonPechinoLista[j][1])
            if(i == k):
                if(point.within(geo_tassellation['geometry'][i])==True):
                    result.append(geo_tassellation['geohash_list'][i])
                    find = 1
                else:
                    i = i+1
                    k=  k-1
            else:        
                if(point.within(geo_tassellation['geometry'][i])==True):
                    result.append(geo_tassellation['geohash_list'][i])
                    find = 1
                elif(point.within(geo_tassellation['geometry'][k])==True):
                    result.append(geo_tassellation['geohash_list'][k])
                    find = 1
                else:
                    i=i+1
                    k= k-1
        if(find == 0):
            result.append("none")    
        j=j+1
    return result

In [21]:
#assegno al dataframe totale le aree
import pygeohash as gh

allBeijing['geohash_area']=allBeijing.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=5), axis=1)


In [22]:
allStart = pd.read_csv('./allPickup.csv')#prova con start
allEnd = pd.read_csv('./allDropoff.csv') #prova end

In [23]:
# ai dataframe inizio e fine assegno le aree
allStart['geohash_area']=allStart.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=5), axis=1)
allEnd['geohash_area']=allEnd.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=5), axis=1)


In [24]:
del allStart['Unnamed: 0']
del allEnd['Unnamed: 0']

In [25]:
allArea = pd.DataFrame(geo_tassellation['geohash_list']) #Dataframe della griglia

In [26]:
#ho bisogno di questa funzione poichè non ho nulla ordinato

def Count(allArea,allStartEnd):
    i = 0
    j=0
    count = 0
    
    allAreaLen= len(allArea['geohash_list'])
    allStartEndLen = len(allStartEnd['geohash_area'])
    result = []
    while(i<allAreaLen):
        j=0
        count =0
        k=len(allStartEnd['geohash_area'])-1
        while(j<allStartEndLen and k>j):
            if(j == k):
                if(allArea['geohash_list'][i]==allStartEnd['geohash_area'][j]):
                    count = count+1
                    j=j+1
                    k=k-1
            else:    
                if(allArea['geohash_list'][i]==allStartEnd['geohash_area'][j]):
                    count = count+1
                if(allArea['geohash_list'][i]==allStartEnd['geohash_area'][k]):
                    count=count+1
                k=k-1
                j=j+1
        result.append(count)
        i=i+1
    return result

#molto più efficiente
def Count1(allArea,allS):
    i = 0
    count = 0
    result = []
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j=0
        areaStart = allS[allS.geohash_area == allArea['geohash_list'][i]]
        result.append(len(areaStart['geohash_area']))
        i=i+1
    return result
    

In [27]:
resultCountStart = Count1(allArea,allStart) #conto numero veicoli in partenza da un'area

In [28]:
resultCountEnd = Count1(allArea,allEnd) #conto numero di veicoli in arrivo in un area

In [29]:
resultCountTot = Count1(allArea,allBeijing) #conto numero di veicoli che transitano in un area per una settimana 

In [30]:
allArea['StartingVehicles']= resultCountStart
allArea['EndingVehicles'] = resultCountEnd
allArea['TotNumberRecordVehicles'] = resultCountTot

In [31]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles
0,wx4es,2,2,437
1,wx4f8,80,79,16142
2,wx5n2,0,0,0
3,wx5r5,0,0,13
4,wx5rx,0,0,0
...,...,...,...,...
1098,wx43m,0,0,83
1099,wx41j,0,0,0
1100,wx4f3,51,56,8447
1101,wx4zk,0,0,0


In [32]:
#funzione che trova la velocità massima di una determinata area i
def SpeedMax(allBeijing,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allBeijing[allBeijing.geohash_area == allArea['geohash_list'][i]]
        speed = area['speed_km']
        result.append(speed.max())
        i=i+1
    return result

In [33]:
resultTotSpeed = SpeedMax(allBeijing,allArea) #calcolo la velocità massima per area

In [34]:
allArea['maxSpeed_km/h'] = resultTotSpeed #aggiungo l'attributo velocità al dataframe

In [35]:
#funzione che mi ritorna la velocità media di una determinata area
def AverageSpeed(allBeijing,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allBeijing[allBeijing.geohash_area == allArea['geohash_list'][i]]
        speed = area['speed_km']
        result.append(speed.mean())
        i=i+1
    return result

In [36]:
resultAverageSpeed= AverageSpeed(allBeijing,allArea)

In [37]:
allArea['AverageSpeed'] = resultAverageSpeed #aggiungo l'attributo velocità media al dataframe

In [38]:
allStart['end'] = allEnd['geohash_area'] #aggiorno il dataframe di inzio con l'attributo end
allEnd['start'] = allStart['geohash_area'] #aggiorno il dataframe di fine con l'attributo start

In [39]:
#cerco di vedere quanti sono i veicoli entranti ed i veicoli uscenti in una determinata area 
#uscenti -> iniziano in un area specificat e terminano la corsa in un'altra area
#entranti -> iniziano la corsa in una certa area e terminano la corsa in una specifica area


def checkOutEdge(allArea,allStart):
    i = 0
    count = 0
    result = []
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j=0
        count=0
        areaStart = allStart[allStart.geohash_area == allArea['geohash_list'][i]]
        if(len(areaStart['geohash_area'])==0):
            result.append(count)
            
        else:
            while(j<len(areaStart['geohash_area'])):
                new =areaStart.reset_index()
                del new['index']
                if(new['geohash_area'][j]!= new['end'][j]):
                    count = count+1
                j=j+1
            result.append(count)
            
        i=i+1
    return result



def checkInEdge(allArea,allEnd):
    i = 0
    count = 0
    result = []
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j = 0
        count = 0
        areaEnd = allEnd[allEnd.geohash_area == allArea['geohash_list'][i]]
        if(len(areaEnd)== 0):
            result.append(count)
        else:
            while(j<len(areaEnd['geohash_area'])):
                new = areaEnd.reset_index()
                del new['index']
                if(new['geohash_area'][j]!= new['start'][j]):
                    count = count+1
                j=j+1
            result.append(count)
        i=i+1
    return result

In [40]:
resultOutEdges = checkOutEdge(allArea,allStart)
resultInEdges = checkInEdge(allArea,allEnd)

In [41]:
#aggiunti attributi archi entranti ed archi uscenti
allArea['OutEdges'] = resultOutEdges
allArea['InEdges'] = resultInEdges

In [42]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges
0,wx4es,2,2,437,87.739920,8.385532,2,2
1,wx4f8,80,79,16142,148.981262,14.313472,55,54
2,wx5n2,0,0,0,NaN,NaN,0,0
3,wx5r5,0,0,13,57.600670,16.654790,0,0
4,wx5rx,0,0,0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...
1098,wx43m,0,0,83,73.252139,19.092300,0,0
1099,wx41j,0,0,0,NaN,NaN,0,0
1100,wx4f3,51,56,8447,118.296888,5.488755,22,27
1101,wx4zk,0,0,0,NaN,NaN,0,0


In [43]:
#creo gli slot temporali per verificare quali sono le ore con maggior numero di veicoli
allBeijing['date_time']=pd.to_datetime(allBeijing['date_time'],format='%Y-%m-%d %H:%M:%S')
allBeijing=allBeijing.assign(session=pd.cut(allBeijing.date_time.dt.hour,
                            [0,6,12,18,23],
                            labels=['Night','Morning','Afternoon','Evening'],
                            include_lowest=True))

In [44]:
#Funzione che mi calcola il numero di punti per slot temporale
def coutHour(allArea,allBeijing,slot):
    i = 0
    result = []
    allAreaLen = len(allArea['geohash_list'])
    allBeijingLen = len(allBeijing['session'])
    while(i<allAreaLen):
        j=0
        area = allBeijing[allBeijing.geohash_area == allArea['geohash_list'][i]]
        area1= area[area.session == slot]
        result.append(len(area1['geohash_area']))
        i=i+1
    return result

In [45]:
resultMorning = coutHour(allArea,allBeijing,'Morning')

In [47]:
resultAfternoon = coutHour(allArea,allBeijing,'Afternoon')

In [48]:
resultEvening = coutHour(allArea,allBeijing,'Evening')

In [49]:
resultNight = coutHour(allArea,allBeijing,'Night')

In [50]:
allBeijing.to_csv('allPechino2.csv')

In [51]:
allArea['VehiclesNight(24-5 am)'] = resultNight
allArea['VehiclesMorning(6-12 am)'] = resultMorning
allArea['VehiclesAfternoon(13-18 pm)'] = resultAfternoon
allArea['VehiclesEvening(19-23 pm)'] =resultEvening

In [52]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),VehiclesAfternoon(13-18 pm),VehiclesEvening(19-23 pm)
0,wx4es,2,2,437,87.739920,8.385532,2,2,153,164,69,51
1,wx4f8,80,79,16142,148.981262,14.313472,55,54,2870,4060,5483,3729
2,wx5n2,0,0,0,NaN,NaN,0,0,0,0,0,0
3,wx5r5,0,0,13,57.600670,16.654790,0,0,0,3,0,10
4,wx5rx,0,0,0,NaN,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1098,wx43m,0,0,83,73.252139,19.092300,0,0,1,14,55,13
1099,wx41j,0,0,0,NaN,NaN,0,0,0,0,0,0
1100,wx4f3,51,56,8447,118.296888,5.488755,22,27,2342,1847,2376,1882
1101,wx4zk,0,0,0,NaN,NaN,0,0,0,0,0,0


In [53]:
allArea.to_csv('Geohash.csv')

In [54]:
allArea['VehiclesAfternoon(13-18 pm)'].sum()

331026

In [55]:
allArea['VehiclesEvening(19-23 pm)'].sum()

235361

In [56]:
allArea['VehiclesNight(24-5 am)'].sum()

203216

In [57]:
allArea['VehiclesMorning(6-12 am)'].sum()

243749

In [58]:
#funzione che mi calcola la densità di punti per area( Area = 4.89*4.89)
def Density(allArea):
    i = 0 
    result = []
    areaLen = len(allArea['TotNumberRecordVehicles'])
    while(i<areaLen):
        count = (allArea['TotNumberRecordVehicles'][i])/(4.89*4.89)
        result.append(count)
        i=i+1
    return result

In [59]:
resultDensity = Density(allArea) #densità punti per area

In [60]:
allArea['Density_n/km^2'] = resultDensity #aggiorno il dataframe con l'attributo denistà

In [61]:
allBeijing['speed_m/s'] = allBeijing['speed_km']/3.6

In [65]:
#cambiare il tipo di oggetto della classe record time
def changetype(allBeijing):
    i = 0
    result = []
    allBeijingLen  = len(allBeijing['IntervalRecordTime_s'])
    while(i<allBeijingLen):
        if(allBeijing['IntervalRecordTime_s'][i]=='sosta' or allBeijing['IntervalRecordTime_s'][i]=='end'):
            result.append(0.0)
        else:
            result.append(float(allBeijing['IntervalRecordTime_s'][i]))
        i=i+1
    return result

In [66]:
resultRecordTime = changetype(allBeijing) #cambio il tipo della colonna

In [67]:
allBeijing['recordTime_float'] = resultRecordTime #aggiorno dataframe pechino

In [68]:
allBeijing['acceleration_m/s^2'] = allBeijing['speed_m/s'] / allBeijing['recordTime_float']

In [69]:
def AccMax(allBeijing,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allBeijing[allBeijing.geohash_area == allArea['geohash_list'][i]]
        acceleration = area['acceleration_m/s^2']
        result.append(acceleration.max())
        i=i+1
    return result

In [70]:
resultAcceleration = AccMax(allBeijing,allArea)

In [71]:
allArea['AccelarationMax'] = resultAcceleration

In [72]:
def AccAverage(allBeijing,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allBeijing[allBeijing.geohash_area == allArea['geohash_list'][i]]
        acceleration = area['acceleration_m/s^2']
        result.append(acceleration.mean())
        i=i+1
    return result

In [73]:
resultAverageAcc = AccAverage(allBeijing,allArea)

In [74]:
allArea['AverageAcceleration_m/s^2'] = resultAverageAcc

In [75]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),VehiclesAfternoon(13-18 pm),VehiclesEvening(19-23 pm),Density_n/km^2,AccelarationMax,AverageAcceleration_m/s^2
0,wx4es,2,2,437,87.739920,8.385532,2,2,153,164,69,51,18.275266,1.608753,0.027443
1,wx4f8,80,79,16142,148.981262,14.313472,55,54,2870,4060,5483,3729,675.055725,20.499622,0.336242
2,wx5n2,0,0,0,NaN,NaN,0,0,0,0,0,0,0.000000,NaN,NaN
3,wx5r5,0,0,13,57.600670,16.654790,0,0,0,3,0,10,0.543658,0.053334,0.012960
4,wx5rx,0,0,0,NaN,NaN,0,0,0,0,0,0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,wx43m,0,0,83,73.252139,19.092300,0,0,1,14,55,13,3.471046,1.395958,0.077596
1099,wx41j,0,0,0,NaN,NaN,0,0,0,0,0,0,0.000000,NaN,NaN
1100,wx4f3,51,56,8447,118.296888,5.488755,22,27,2342,1847,2376,1882,353.252119,8.856833,0.080433
1101,wx4zk,0,0,0,NaN,NaN,0,0,0,0,0,0,0.000000,NaN,NaN



Provo ad inserire i punti di intresse che esporto da overpass e li raggruppo per categoria


In [76]:
#prova con il numero di interessi
museum=gpd.read_file('./../data-set/Pechino/Interessi/museum.geojson')
attraction = gpd.read_file('./../data-set/Pechino/Interessi/attraction.geojson')
univerisy=gpd.read_file('./../data-set/Pechino/Interessi/university.geojson')
bank =  gpd.read_file('./../data-set/Pechino/Interessi/bank.geojson')
hospital=gpd.read_file('./../../../Scaricati/export (2).geojson')
dentist=gpd.read_file('./../data-set/Pechino/Interessi/dentist.geojson')
cinema = gpd.read_file('./../data-set/Pechino/Interessi/cinema.geojson')
stadium = gpd.read_file('./../data-set/Pechino/Interessi/stadium.geojson')
fuel = gpd.read_file('./../data-set/Pechino/Interessi/fuel-station.geojson')
zoo = gpd.read_file('./../data-set/Pechino/Interessi/zoo.geojson')
pharmacy = gpd.read_file('./../data-set/Pechino/Interessi/pharmacy.geojson')
busStation = gpd.read_file('./../data-set/Pechino/Interessi/BusStation.geojson')
ruins = gpd.read_file('./../data-set/Pechino/Interessi/ruins.geojson')
swimmingPool = gpd.read_file('./../data-set/Pechino/Interessi/swimmingPool.geojson')
taxi_stop = gpd.read_file('./../data-set/Pechino/Interessi/taxi_stops.geojson')
company = gpd.read_file('./../data-set/Pechino/Interessi/company.geojson')
library = gpd.read_file('./../data-set/Pechino/Interessi/library.geojson')
theatre = gpd.read_file('./../data-set/Pechino/Interessi/theatre.geojson')

In [77]:
def checkInterest(allArea,interest):
    i = 0
    count = 0
    result = []
    interestLen = len(interest['geometry'])
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j = 0
        count = 0
        while(j<interestLen):
            if(interest['geometry'][j].centroid.within(allArea['geometry'][i])):
                count = count+1
            
            j=j+1
        result.append(count)    
        i=i+1
    return result

In [78]:
type(bank['geometry'][0].centroid)

shapely.geometry.point.Point

In [79]:
resultMuseum = checkInterest(geo_tassellation,museum)
resultAttraction = checkInterest(geo_tassellation,attraction)
resultUniversity= checkInterest(geo_tassellation,univerisy)
resultBank = checkInterest(geo_tassellation,bank)
resultHospital = checkInterest(geo_tassellation,hospital)
resultDentist = checkInterest(geo_tassellation,dentist)
resultCinema = checkInterest(geo_tassellation,cinema)
resultStadium = checkInterest(geo_tassellation,stadium)
resultPharmacy = checkInterest(geo_tassellation,pharmacy)
resultFuel = checkInterest(geo_tassellation,fuel)
resultZoo = checkInterest(geo_tassellation,zoo)
resultBus = checkInterest(geo_tassellation,busStation)
resultRuins = checkInterest(geo_tassellation,ruins)
resultSP = checkInterest(geo_tassellation,swimmingPool)
resultTaxi = checkInterest(geo_tassellation,taxi_stop)
resultCompany = checkInterest(geo_tassellation,company)
resultLibrary = checkInterest(geo_tassellation,library)
resultTheatre = checkInterest(geo_tassellation,theatre)

In [80]:
resultCulture= []
i = 0

while(i<len(resultAttraction)):
    count = 0
    count = resultMuseum[i]+ resultAttraction[i]+resultRuins[i]
    resultCulture.append(count)
    i=i+1


In [81]:
resultHealth= []
i =0

while(i<len(resultHospital)):
    count = 0
    count = resultHospital[i] + resultDentist[i] + resultPharmacy[i]
    resultHealth.append(count)
    i=i+1

In [82]:
resultFinance= []
i =0

while(i<len(resultBank)):
    count = 0
    count = resultBank[i] + resultCompany[i]
    resultFinance.append(count)
    i=i+1

In [83]:
resultEnterteiment= []
i =0

while(i<len(resultCinema)):
    count = 0
    count = resultCinema[i] + resultTheatre[i] + resultZoo[i] + resultStadium[i] +resultSP[i]
    resultEnterteiment.append(count)
    i=i+1

In [84]:
resultTrasportation = []
i = 0
while(i<len(resultBus)):
    count = 0
    count = resultBus[i] +resultFuel[i] + resultTaxi[i]
    resultTrasportation.append(count)
    i = i+1

In [85]:
resultEducation = []
i = 0
while(i<len(resultLibrary)):
    count = 0
    count = resultUniversity[i] + resultLibrary[i]
    resultEducation.append(count)
    i=i+1

In [86]:
allArea['NumberCulturalPoint("Museum-Attraction-Ruins")'] = resultCulture
allArea['NumberFinancePoint("Bank-Company")'] = resultBank
allArea['NumberHealthPoint("Hospital-Dentist-Pharmacy")'] = resultHealth
allArea['NumberTrasportationPoint("Fuel-BusStation-TaxiStop")'] = resultTrasportation
allArea['NumberEducationPoint("University-Library")'] = resultEducation
allArea['NumberEnterteimentPoint("Cinema-Theatre-Zoo-Stadium-SwimmingPool")'] = resultEnterteiment

Manca da verificare il numero di punti durante i giorni della settimana (fino a venerdì) ed il weekend.

I record che sono registrati fanno riferimento ai giorni di febbraio dal 02 al 08. Il 2 di febbraio era sabato,quindi arriverò fino a venerdì 08

In [89]:
import datetime as dt
prova= allBeijing[allBeijing['date_time'].dt.year==2008]
prova= prova[prova['date_time'].dt.month==2]
prova= prova[prova['date_time'].dt.day==2]

In [90]:
#creo 7 dataframe per i giorni della settimana

#02
day2= allBeijing[allBeijing['date_time'].dt.year==2008]
day2= day2[day2['date_time'].dt.month==2]
day2= day2[day2['date_time'].dt.day==2]

#03
day3= allBeijing[allBeijing['date_time'].dt.year==2008]
day3= day3[day3['date_time'].dt.month==2]
day3= day3[day3['date_time'].dt.day==3]

#04
day4= allBeijing[allBeijing['date_time'].dt.year==2008]
day4= day4[day4['date_time'].dt.month==2]
day4= day4[day4['date_time'].dt.day==4]

#05
day5= allBeijing[allBeijing['date_time'].dt.year==2008]
day5= day5[day5['date_time'].dt.month==2]
day5= day5[day5['date_time'].dt.day==5]

#06
day6= allBeijing[allBeijing['date_time'].dt.year==2008]
day6= day6[day6['date_time'].dt.month==2]
day6= day6[day6['date_time'].dt.day==6]

#07
day7= allBeijing[allBeijing['date_time'].dt.year==2008]
day7= day7[day7['date_time'].dt.month==2]
day7= day7[day7['date_time'].dt.day==7]

#08
day8= allBeijing[allBeijing['date_time'].dt.year==2008]
day8= day8[day8['date_time'].dt.month==2]
day8= day8[day8['date_time'].dt.day==8]


In [93]:
resultday2 = Count1(allArea,day2)
resultday3 = Count1(allArea,day3)
resultday4 = Count1(allArea,day4)
resultday5 = Count1(allArea,day5)
resultday6 = Count1(allArea,day6)
resultday7 = Count1(allArea,day7)
resultday8 = Count1(allArea,day8)


In [95]:
#calcolo il numero di punti nella settimana e nel weekend 
i=0
resultWeekEnd= []
while(i<len(resultday2)):
    count = 0
    count = resultday2[i]+resultday3[i]
    resultWeekEnd.append(count)
    i=i+1

    

i=0
resultWeek= []
while(i<len(resultday4)):
    count = 0
    count = resultday4[i]+resultday5[i] +resultday6[i] + resultday7[i] + resultday8[i]
    resultWeek.append(count)
    i=i+1

In [96]:
allArea['NumberWeekPoint(4-5-6-7-8)'] = resultWeek
allArea['NumberWeekendPoint(2-3)'] = resultWeekEnd

In [97]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),...,AccelarationMax,AverageAcceleration_m/s^2,"NumberCulturalPoint(""Museum-Attraction-Ruins"")","NumberFinancePoint(""Bank-Company"")","NumberHealthPoint(""Hospital-Dentist-Pharmacy"")","NumberTrasportationPoint(""Fuel-BusStation-TaxiStop"")","NumberEducationPoint(""University-Library"")","NumberEnterteimentPoint(""Cinema-Theatre-Zoo-Stadium-SwimmingPool"")",NumberWeekPoint(4-5-6-7-8),NumberWeekendPoint(2-3)
0,wx4es,2,2,437,87.739920,8.385532,2,2,153,164,...,1.608753,0.027443,5,2,2,3,1,0,340,97
1,wx4f8,80,79,16142,148.981262,14.313472,55,54,2870,4060,...,20.499622,0.336242,2,1,5,8,0,3,11349,4793
2,wx5n2,0,0,0,NaN,NaN,0,0,0,0,...,NaN,NaN,0,0,0,0,0,0,0,0
3,wx5r5,0,0,13,57.600670,16.654790,0,0,0,3,...,0.053334,0.012960,0,0,0,0,0,0,13,0
4,wx5rx,0,0,0,NaN,NaN,0,0,0,0,...,NaN,NaN,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,wx43m,0,0,83,73.252139,19.092300,0,0,1,14,...,1.395958,0.077596,0,0,0,0,0,0,47,36
1099,wx41j,0,0,0,NaN,NaN,0,0,0,0,...,NaN,NaN,0,0,0,0,0,0,0,0
1100,wx4f3,51,56,8447,118.296888,5.488755,22,27,2342,1847,...,8.856833,0.080433,1,2,1,3,0,1,7075,1372
1101,wx4zk,0,0,0,NaN,NaN,0,0,0,0,...,NaN,NaN,0,0,0,0,0,0,0,0


In [98]:
allArea.to_csv('Geohash1.csv')

In [99]:
#mi sono scordato il numero totale di punti di interesse

i = 0
totalInterest = []
while(i<len(resultBus)):
    count = 0
    count = resultCulture[i]+ resultFinance[i]+resultEnterteiment[i]+resultHealth[i]+resultTrasportation[i]+resultEducation[i]
    totalInterest.append(count)
    i=i+1


In [100]:
allArea['NumberTotalPointInterest'] = totalInterest

In [101]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),...,AverageAcceleration_m/s^2,"NumberCulturalPoint(""Museum-Attraction-Ruins"")","NumberFinancePoint(""Bank-Company"")","NumberHealthPoint(""Hospital-Dentist-Pharmacy"")","NumberTrasportationPoint(""Fuel-BusStation-TaxiStop"")","NumberEducationPoint(""University-Library"")","NumberEnterteimentPoint(""Cinema-Theatre-Zoo-Stadium-SwimmingPool"")",NumberWeekPoint(4-5-6-7-8),NumberWeekendPoint(2-3),NumberTotalPointInterest
0,wx4es,2,2,437,87.739920,8.385532,2,2,153,164,...,0.027443,5,2,2,3,1,0,340,97,13
1,wx4f8,80,79,16142,148.981262,14.313472,55,54,2870,4060,...,0.336242,2,1,5,8,0,3,11349,4793,19
2,wx5n2,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
3,wx5r5,0,0,13,57.600670,16.654790,0,0,0,3,...,0.012960,0,0,0,0,0,0,13,0,0
4,wx5rx,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,wx43m,0,0,83,73.252139,19.092300,0,0,1,14,...,0.077596,0,0,0,0,0,0,47,36,0
1099,wx41j,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
1100,wx4f3,51,56,8447,118.296888,5.488755,22,27,2342,1847,...,0.080433,1,2,1,3,0,1,7075,1372,8
1101,wx4zk,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0


In [102]:
allArea.to_csv('Geohash1.csv')

In [1]:
import pandas as pd
allArea = pd.read_csv('Geohash1.csv')

In [3]:
allArea.head(50)

,Unnamed: 0,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),...,AverageAcceleration_m/s^2,"NumberCulturalPoint(""Museum-Attraction-Ruins"")","NumberFinancePoint(""Bank-Company"")","NumberHealthPoint(""Hospital-Dentist-Pharmacy"")","NumberTrasportationPoint(""Fuel-BusStation-TaxiStop"")","NumberEducationPoint(""University-Library"")","NumberEnterteimentPoint(""Cinema-Theatre-Zoo-Stadium-SwimmingPool"")",NumberWeekPoint(4-5-6-7-8),NumberWeekendPoint(2-3),NumberTotalPointInterest
0,0,wx4es,2,2,437,87.739920,8.385532,2,2,153,...,0.027443,5,2,2,3,1,0,340,97,13
1,1,wx4f8,80,79,16142,148.981262,14.313472,55,54,2870,...,0.336242,2,1,5,8,0,3,11349,4793,19
2,2,wx5n2,0,0,0,NaN,NaN,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
3,3,wx5r5,0,0,13,57.600670,16.654790,0,0,0,...,0.012960,0,0,0,0,0,0,13,0,0
4,4,wx5rx,0,0,0,NaN,NaN,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
5,5,wx54b,4,4,78,80.996553,23.273664,1,1,14,...,0.136625,0,0,0,1,0,0,65,13,1
6,6,wx5hu,0,0,17,100.904538,16.157099,0,0,14,...,0.346583,0,0,0,0,0,0,17,0,0
7,7,wx4ds,9,10,2706,95.640036,21.769447,5,6,433,...,2.040624,3,0,0,5,0,0,2359,347,8
8,8,wx4kj,0,0,0,NaN,NaN,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
9,9,wx4mj,0,0,0,NaN,NaN,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
